In [1]:
import os
import sys
import csv
import codecs
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import scipy.stats as stats
import seaborn as sns
import random
import copy
from itertools import combinations

import pandas as pd

# Open data

In [2]:
data_files = { "p1_part1" : "..\\Test_results\\Results_SynDet_part1_participant1.csv",
             "p1_part2" : "..\\Test_results\\Results_SynDet_part2_participant1.csv",
             "p2_part1" : "..\\Test_results\\Results_SynDet_part1_participant2.csv",
              "p2_part2" : "..\\Test_results\\Results_SynDet_part2_participant2.csv",
              "p3_part1" : "..\\Test_results\\Results_SynDet_part1_participant3.csv",
              "p3_part2" : "..\\Test_results\\Results_SynDet_part2_participant3.csv"
             }

In [3]:
freq_list = []
with open("..\\Test_results\\Frequencies.csv", 'r', encoding="utf8") as csvfile:
        reader = csv.reader(csvfile, delimiter = ";")
        for line in reader:
            freq_list.append(line[0])

In [4]:
def create_data_list(file, list_name):
    with open(file, 'r', encoding="utf8") as csvfile:
        reader = csv.reader(csvfile, delimiter = ";")
        count = 0
        for n, N2V_additive, N2V_Standard, N2V_Consistency, W2V, SVD_All, SVD_Quine, Non, Rank, Comment in reader:
            if count > 0:
                list_name.append([int(n), int(N2V_additive), int(N2V_Standard), int(N2V_Consistency), int(W2V), int(SVD_All), int(SVD_Quine), int(Non), int(Rank), Comment])
            count += 1

In [5]:
p1 = []
p2 = []
p3 = []
for n, (name, file) in enumerate(data_files.items()):
    if n < 2: 
        create_data_list(file, p1)
    elif n < 4:
        create_data_list(file, p2)
    else:
        create_data_list(file, p3)

In [6]:
for n in range(len(freq_list)):
    p1[n].append(freq_list[n])
    p2[n].append(freq_list[n])
    p3[n].append(freq_list[n])

In [7]:
results_vector_p1 = []
results_vector_p2 = []
results_vector_p3 = []
for x, row in enumerate(p1):
    bool = False
    for n, item in enumerate(row):
        if 0 < n < 7 and item == 1:
            if not bool: 
                results_vector_p1.append(str(n))
            else:
                results_vector_p1[x] += str(n)
            bool = True
        elif n == 7 and not bool :
            results_vector_p1.append(str(0))
for x, row in enumerate(p2):
    bool = False
    for n, item in enumerate(row):
        if 0 < n < 7 and item == 1:
            if not bool: 
                results_vector_p2.append(str(n))
            else:
                results_vector_p2[x] += str(n)
            bool = True
        elif n == 7 and not bool :
            results_vector_p2.append(str(0))

for x, row in enumerate(p3):
    bool = False
    for n, item in enumerate(row):
        if 0 < n < 7 and item == 1:
            if not bool: 
                results_vector_p3.append(str(n))
            else:
                results_vector_p3[x] += str(n)
            bool = True
        elif n == 7 and not bool :
            results_vector_p3.append(str(0))

#for the calculation of inter rater agreement we can represent the words returned by multiple models 
#by just one model
results_vector_p1 = [x[0] for x in results_vector_p1]
results_vector_p2 = [x[0] for x in results_vector_p2]

In [8]:
#lists of high and low frequency words per participant
p1_high = [model for (n, model) in enumerate(results_vector_p1) if p1[n][-1] == 'high']
p1_low = [model for (n, model) in enumerate(results_vector_p1) if p1[n][-1] == 'low']
p2_high = [model for (n, model) in enumerate(results_vector_p2) if p2[n][-1] == 'high']
p2_low = [model for (n, model) in enumerate(results_vector_p2) if p2[n][-1] == 'low']

#lists of high and low rank words per participant
p1_rhigh = [model for (n, model) in enumerate(results_vector_p1) if p1[n][-3] == 2]
p1_rlow = [model for (n, model) in enumerate(results_vector_p1) if p1[n][-3] == 10]
p2_rhigh = [model for (n, model) in enumerate(results_vector_p2) if p2[n][-3] == 2]
p2_rlow = [model for (n, model) in enumerate(results_vector_p2) if p2[n][-3] == 10]


# Inter rater agreement scores

In [9]:
from sklearn.metrics import cohen_kappa_score
print("inter rater agreement as measured by Cohen's kappa between participants:")
print("p1 p2 k=", cohen_kappa_score(results_vector_p1, results_vector_p2))
print("p1 p3 k=",cohen_kappa_score(results_vector_p1, results_vector_p3))
print("p2 p3 k=",cohen_kappa_score(results_vector_p2, results_vector_p3))

inter rater agreement as measured by Cohen's kappa between participants:
p1 p2 k= 0.48349887856456264
p1 p3 k= 0.18570555894929752
p2 p3 k= 0.16792144026186573


In [10]:
def joint_probability(ann1_labels, ann2_labels):
        zipped = zip(ann1_labels, ann2_labels)
        agree = [1 if label[0] == label[1] else 0 for label in zipped]
        return sum(agree) / len(agree)

In [11]:
jb = joint_probability(p1_high, p2_high)
print("joint probability high frequency words: ", jb)
jb = joint_probability(p1_low, p2_low)
print("joint probability low frequency words: ", jb)
ck_high = cohen_kappa_score(p1_high, p2_high)
print("high frequency words Cohen's Kappa: ", ck_high)
ck_low = cohen_kappa_score(p1_low, p2_low)
print("low frequency words Cohen's Kappa: ", ck_low)
print()
jb = joint_probability(p1_rhigh, p2_rhigh)
print("joint probability high rank: ", jb)
jb = joint_probability(p1_rlow, p2_rlow)
print("joint probability low rank: ", jb)
ck_rhigh = cohen_kappa_score(p1_rhigh, p2_rhigh)
print("high rank Cohen's Kappa: ", ck_rhigh)
ck_rlow = cohen_kappa_score(p1_rlow, p2_rlow)
print("low rank Cohen's Kappa: ", ck_rlow)

joint probability high frequency words:  0.5666666666666667
joint probability low frequency words:  0.59375
high frequency words Cohen's Kappa:  0.4205052005943536
low frequency words Cohen's Kappa:  0.5123094958968347

joint probability high rank:  0.6129032258064516
joint probability low rank:  0.5483870967741935
high rank Cohen's Kappa:  0.4545454545454545
low rank Cohen's Kappa:  0.44501278772378516


In [12]:
def joint_nozero(ann1, ann2):
    zipped = zip(ann1, ann2)
    agree = []
    for n, label in enumerate(zipped):
        if label [0] != '0' and label[1] != '0':
            if label[0] == label[1]:
                agree.append(1)
            else:
                agree.append(0)
    return sum(agree) / len(agree)

def joint_zero(ann1, ann2):
        zipped = zip(ann1, ann2)
        agree = []
        for label in zipped:
            if label[0] == '0' or label[1] == '0':
                if label[0] == label[1]:
                    agree.append(1)
                else:
                    agree.append(0)
        return sum(agree) / len(agree)

In [13]:
jb = joint_probability(results_vector_p1, results_vector_p2)#df, 1,2)
print("general joint probability : ", jb)
jb = joint_zero(results_vector_p1, results_vector_p2)
print("joint probability for the none option:", jb)
jb = joint_nozero(results_vector_p1, results_vector_p2)
print("joint probability for the not-none option:", jb)
print()
zipped = zip(results_vector_p1, results_vector_p2)
zero_1, zero_2 = [], []
no_zero_1, no_zero_2 = [], []
for label in zipped:
    if label[0] == '0' or label[1] == '0':
        zero_1.append(label[0])
        zero_2.append(label[1])
    else:
        no_zero_1.append(label[0])
        no_zero_2.append(label[1])
ck_general = cohen_kappa_score(results_vector_p1, results_vector_p2)
print("general Cohen's Kappa: ", ck_general)
ck_zero = cohen_kappa_score(zero_1, zero_2)
print("Cohen's Kappa for the none option: ", ck_zero)
ck_no_zero = cohen_kappa_score(no_zero_1, no_zero_2)
print("Cohen's Kappa for the not-none option: ", ck_no_zero)

general joint probability :  0.5806451612903226
joint probability for the none option: 0.2727272727272727
joint probability for the not-none option: 0.75

general Cohen's Kappa:  0.48349887856456264
Cohen's Kappa for the none option:  -0.11041009463722395
Cohen's Kappa for the not-none option:  0.6566523605150214


# Most and least agreed words

In [14]:
ia_path = '..\\Test_results\\syn_det_results-word-freqs.csv'
ia_df = pd.read_csv(ia_path, sep=';')
ia_df.head()

,Unnamed: 0,p1,p2,freq,word
0,0,6,6,high,objects
1,1,3,3,low,memory
2,2,0,3,low,intensional_abstraction
3,3,4,4,low,intensional_abstraction
4,4,12,0,high,description


In [15]:
agreed_words = list(ia_df[ia_df['p1'] ==ia_df['p2']]['word'].values)
print("Word that was agreed upon each time:")
for word in agreed_words:
    #each word is tested on two ranks, so two agreements is the maximum
    if agreed_words.count(word) == 2:
        print(agreed_words.count(word), word)

Word that was agreed upon each time:
2 paradox
2 nouns
2 subtraction
2 truth_vehicles
2 nouns
2 intension
2 subtraction
2 mental_states
2 utterances
2 paradox
2 mental_states
2 intension
2 object
2 object
2 quantification
2 quantification
2 information
2 utterances
2 information
2 truth_vehicles


In [16]:
disagreed_words = list(ia_df[ia_df['p1'] !=ia_df['p2']]['word'].values)
print("Word that was disagreed upon each time:")
for word in disagreed_words:
    #each word is tested on two ranks, so two disagreements is the maximum
    if disagreed_words.count(word) == 2:
        print(disagreed_words.count(word), word)
        

Word that was disagreed upon each time:
2 attributes
2 application
2 application
2 transparency
2 adjectives
2 attributes
2 logical_particles
2 adjectives
2 transparency
2 logical_particles
